In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.append('src')  
from claude import condition_claude

condition_claude()

API key is set. Proceeding with normal operation.
Provide instructions in conversational text, and a LaTeX template for the analysis output


In [4]:
import ipywidgets as widgets
from IPython.display import display

# Create an Output widget to capture and display output dynamically
output = widgets.Output()

def on_submit_button_clicked(b):
    with output:
        output.clear_output()
        print("Instructions Content:", f'{instructions_textarea.value[:40]}...')
        print("Template Content:", f'{template_textarea.value[:40]}...')


template_textarea = widgets.Textarea(
    value='',
    placeholder='Enter LaTeX template...',
    description='Template:',
    disabled=False,
    layout={'width': '100%', 'height': '300px'}
)

instructions_textarea = widgets.Textarea(
    value='',
    placeholder='Enter written instructions...',
    description='Instructions:',
    disabled=False,
    layout={'width': '100%', 'height': '300px'}
)


submit_button = widgets.Button(
    description='Submit to Claude',
    button_style='success',
    tooltip='Click to submit'
)

output = widgets.Output()

# Link the event handler function to the button
submit_button.on_click(on_submit_button_clicked)

# Arrange the text areas horizontally
text_areas_hbox = widgets.HBox([instructions_textarea,template_textarea])

# Display everything
display(text_areas_hbox, submit_button, output)



Button(button_style='success', description='Submit to Claude', style=ButtonStyle(), tooltip='Click to submit')

Output()

Textarea(value='\\begin{description}\n    \\item[Effects on {DV}] Analysis revealed ...\n        \\paragraph{Main effects}\n            \\begin{itemize}\n                \\item {list f size and marginal means for each IV}.\n                \\item {IV2....}\n            \\end{itemize}\n        \\paragraph{Two-way interactions}\n            \\begin{itemize}\n                \\item {analysis on two-way interactions, if any...}\n                \\item {etc..\n            \\end{itemize}\n\n            {Final analysis summary in 1 or 2 sentences}\\\\\n\\end{description}', description='Template:', layout=Layout(height='300px', width='80%'), placeholder='..replace with LaTEX')


```tex
\begin{description}
    \item[Effects on {DV}] Analysis revealed ...
        \paragraph{Main effects}
            \begin{itemize}
                \item {list f size and marginal means for each IV}.
                \item {IV2....}
            \end{itemize}
        \paragraph{Two-way interactions}
            \begin{itemize}
                \item {analysis on two-way interactions, if any...}
                \item {etc..
            \end{itemize}

            {Final analysis summary in 1 or 2 sentences}\\
\end{description}
```

In [3]:
import os
import pandas as pd
from pyvttbl import DataFrame
from pyvttbl.stats import Anova
import numpy as np
import pprint

# Define study variables
# Set to True for FATR study, False for Depend study
STUDY_FATR = False  
STUDY_FATR = True
# True for human-readable output, False for structured data
HUMAN_READABLE = True  
HUMAN_READABLE = False

## Set DVs >> process multiple in one list, or just one at a time seems easier... 
DV = 'Diagnosis Time'
DV = 'Cooper Harper'
DV = 'NasaTLX'
DV = 'Decision'

# Paths and filenames
DATA_DIR = 'data'
TEMP_CSV_PATH = 'temp_cleaned_data.csv'

# Determine study based on STUDY_FATR flag
if STUDY_FATR:
    print("** FATR Study **")
    source_filename = 'fatr_students.csv'
    between_factor = "Study"
else:
    print("** Depend Study **")
    source_filename = 'depend_only.csv'
    between_factor = "Expertise"

source_path = os.path.join(DATA_DIR, source_filename)

# Load and preprocess data
df_pandas = pd.read_csv(source_path)

if DV == "NasaTLX":
    print("Creating NasaTLX composite measure...")
    # Invert 'Performance' scores and fill missing values with median
    df_pandas['Performance'] = 11 - df_pandas['Performance']
    columns = ['Performance', 'Temporal Demand', 'Frustration Level', 'Mental Demand', 'Effort']
    for column in columns:
        median = df_pandas[column].median()
        df_pandas[column].fillna(median, inplace=True)
    # Calculate the NasaTLX composite score
    df_pandas['NasaTLX'] = df_pandas[columns].mean(axis=1)
else:
    # Drop rows with missing values in the DV
    df_pandas.dropna(subset=[DV], inplace=True)

# Save cleaned data to temporary CSV
df_pandas.to_csv(TEMP_CSV_PATH, index=False)

# Load data into pyvttbl DataFrame for ANOVA
df_pyvttbl = DataFrame()
df_pyvttbl.read_tbl(TEMP_CSV_PATH)

# ANOVA settings
wfactors = ['Scenario', 'Fault']
bfactors = [between_factor]

# Run ANOVA
print(f"Running ANOVA for: {DV}")
aov = Anova()
aov.run(df_pyvttbl, DV, wfactors=wfactors, bfactors=bfactors)
aov.truncate(test='gg')

# Human-readable output
if HUMAN_READABLE:
    print(f'{aov}\n\n')

aov_dict_str = str(dict(aov))
marginal_means_str = str(aov.get_marginal_means(['Scenario', 'Fault', between_factor]))

# Combine the two strings with a separator for clarity
combined_summary = "ANOVA Results:\n" + aov_dict_str + "\n\nMarginal Means:\n" + marginal_means_str


# These variables now hold the raw data in whatever format they originally had
# For example, aov_dict could be a dictionary with various statistics,
# and marginal_means could be a pandas DataFrame or a dictionary

def print_partial_dict(d, limit=1):
    """Print a limited number of key-value pairs from a dictionary."""
    for i, (key, value) in enumerate(d.items()):
        if i >= limit: break
        print(f"{key}: {value}")

def print_partial_list(lst, limit=1):
    """Print a limited number of items from a list or tuple."""
    for item in lst[:limit]:
        print(item)


if not HUMAN_READABLE:
    print("\n** Partial AOV Dictionary:")
    if isinstance(aov_dict, dict):
        print_partial_dict(aov_dict)
    elif isinstance(aov_dict, (list, tuple)):
        print_partial_list(aov_dict)
    else:
        print(aov_dict)  # Fallback if it's neither a dict nor a list/tuple
    
    print("\n** Partial Marginal Means:")
    if isinstance(marginal_means, dict):
        print_partial_dict(marginal_means)
    elif isinstance(marginal_means, (list, tuple)):
        print_partial_list(marginal_means)
    else:
        print(marginal_means)  # Fallback if it's neither a dict nor a list/tuple




** FATR Study **
Running ANOVA for: Decision
('Scenario',)
('Study',)
('Fault', 'Study')
('Scenario', 'Fault', 'Study')

** Partial AOV Dictionary:


NameError: name 'aov_dict' is not defined